In [81]:
import os
import re
import json
import random
import sys
import asyncio
import pickle
import datetime
import gurobipy as gp
sys.path.append('../')

from openai import OpenAI, AsyncClient
from json import JSONDecodeError
from enum import Enum
from tqdm.auto import tqdm
from utils import *
from pydantic import BaseModel
from colorama import Fore, Style
from llama_index.program.openai import OpenAIPydanticProgram
from llama_index.llms.openai import OpenAI

In [82]:
cfg = json.load(open('../configs./configs.json', 'r'))
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
asyncclient = AsyncClient(api_key=os.environ["OPENAI_API_KEY"])

dt = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')

In [83]:
DATA_DIR = '../data'
DATASET_NAME = 'LPWP' 
OUTPUT_DIR = '../output'  

nl4opt_data = read_txt_file(os.path.join(DATA_DIR, DATASET_NAME, 'lpwp.txt'))
questions, answers = get_nl4opt_qas(nl4opt_data)
assert len(questions) == len(answers)

qa_pairs = list(zip(questions, answers))
# demo_samples, test_samples = get_demo_and_test_samples(qa_pairs)

questions = [q for q, _ in qa_pairs]
answers = [a for _, a in qa_pairs]

2024-09-19 16:50:43.842 | DEBUG    | utils:read_txt_file:15 - Reading file: ../data\LPWP\lpwp.txt
2024-09-19 16:50:43.844 | DEBUG    | utils:read_txt_file:17 - File read successfully: ../data\LPWP\lpwp.txt
2024-09-19 16:50:43.845 | INFO     | utils:get_nl4opt_qas:36 - Number of questions: 288
2024-09-19 16:50:43.845 | INFO     | utils:get_nl4opt_qas:37 - Number of answers: 288


In [84]:
# class VarType(Enum):
#     INTEGER = "INTEGER"
#     CONTINUOUS = "CONTINUOUS"

class CodeScheme(BaseModel):
    """The variable type in the problem and the code string"""
    
    # var_type: VarType
    code: str     

In [85]:
sys_prompt = """You are an expert in optimization problems and domain specific language generation. 
Your task is to convert the textual optimization text into lines of code.
You should also analyze whether the variable in the optimization problem should be INTEGER or CONTINUOUS.
DO NOT ADD ANY COMMENTS OR EXPLANATION TO THE CODE. JUST OUTPUT THE CODE.
Here are some examples that you should refer to:\n"""

example = """
QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
CODE:
x = m.addVar(name="Oil Max", vtype=gp.GRB.INTEGER)
y = m.addVar(name="Oil Max Pro", vtype=gp.GRB.INTEGER)
m.setObjective(10 * x + 15 * y, gp.GRB.MAXIMIZE)
m.addConstr(46 * x + 13 * y <= 1345)
m.addConstr(43 * x + 4 * y <= 346)
m.addConstr(56 * x + 45 * y <= 1643)

QUESTION:
Ben is growing apples and pears on his orchard. He has 50 acres available on which he must grow a minimum of 5 acres of apples and a minimum of 10 acres of pears to meet demands. The profit per apple is $2 and the profit per pear is $4. He prefers to grow more pears than apples but limitations in his workforce allow him to grow at most twice the amount of pears as apples. How many of each fruit should Ben grow in order to maximize his profit? What is that profit?
CODE:
x = m.addVar(name="apples", vtype=gp.GRB.INTEGER)
y = m.addVar(name="pears", vtype=gp.GRB.INTEGER)
m.setObjective(2 * x + 4 * y, gp.GRB.MAXIMIZE)
m.addConstr(x + y <= 50)
m.addConstr(x >= 5)
m.addConstr(y >= 10)
m.addConstr(y <= 2 * x)
"""

prompt_template_str = sys_prompt + example + "\nPlease finish the task think step by step.\nQUESTION:{q}"
print(prompt_template_str)

You are an expert in optimization problems and domain specific language generation. 
Your task is to convert the textual optimization text into lines of code.
You should also analyze whether the variable in the optimization problem should be INTEGER or CONTINUOUS.
DO NOT ADD ANY COMMENTS OR EXPLANATION TO THE CODE. JUST OUTPUT THE CODE.
Here are some examples that you should refer to:

QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
CODE

In [86]:
program = OpenAIPydanticProgram.from_defaults(
    output_cls=CodeScheme, 
    llm=OpenAI("gpt-4"),
    prompt_template_str=prompt_template_str.format(q=questions[2]), 
    verbose=True
)

In [87]:
code = program()

Function call: CodeScheme with args: {
  "code": "x = m.addVar(name=\"Fertilizer A\", vtype=gp.GRB.CONTINUOUS)\ny = m.addVar(name=\"Fertilizer B\", vtype=gp.GRB.CONTINUOUS)\nm.setObjective(5 * x + 9 * y, gp.GRB.MINIMIZE)\nm.addConstr(13 * x + 8 * y >= 220)\nm.addConstr(5 * x + 14 * y >= 160)\nm.addConstr(6 * x + 6 * y <= 350)"
}


In [88]:
execute_code(complement_code(clean_code(code.code)))

125.49295774647888

In [89]:
# batch_size = 8
# lp_reasoning_list = []
# for idx in tqdm(range(0, len(questions), batch_size)):
#     batch = questions[idx:idx+batch_size]
    
#     tasks = [asyncclient.beta.chat.completions.parse(
#         model="gpt-4o-2024-08-06",
#         temperature=0,
#         response_format=Code,
#         messages=[
#             {"role": "system", "content": sys_prompt},
#             {"role": "user", "content": f"QUESTION: {q}"}
#         ]) for q in batch
#     ]

#     combined_responses = await asyncio.gather(*tasks)
#     lp_reasoning_list.extend([r.choices[0].message.parsed for r in combined_responses])

codes = []
for i in tqdm(range(len(questions))):
    program = OpenAIPydanticProgram.from_defaults(
        output_cls=CodeScheme, 
        llm=OpenAI("gpt-4"),
        prompt_template_str=prompt_template_str.format(q=questions[i]), 
        verbose=False
    )
    code = program()
    codes.append(code)
    
# if len(codes) >= save_chunk_size:
#     chunk_count += 1
#     loguru.logger.debug(f"Saving chunk {chunk_count}, chunk size {len(codes)}")
#     filename = 'e2e_codegen_gpt_4_lpwp_gurobi_' + str(chunk_count) + '.pkl'
#     with open(os.path.join(OUTPUT_DIR, filename), 'wb') as f:
#         pickle.dump(codes, f)
#     codes = []

100%|██████████| 288/288 [33:59<00:00,  7.08s/it]


In [90]:
filename = 'e2e_codegen_gpt_4_lpwp_gurobi_' + dt + '.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'wb') as f:
    pickle.dump(codes, f)

In [92]:
filename = 'e2e_codegen_gpt_4_lpwp_gurobi_2024-09-19-16-50-41.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'rb') as f:
    codes = pickle.load(f)

In [93]:
code_strs = [c.code for c in codes]

In [94]:
len(code_strs)

288

In [95]:
prefix = """
import gurobipy as gp
env = gp.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()
m = gp.Model(env=env)
"""
                
suffix = """
m.optimize()
"""

def complement_code(code: str) -> float:
    return prefix + code + suffix

In [96]:
def clean_code(code: str) -> str:
    cleand_code = []
    for line in code.split('\n'):
        line = line.strip()
        if line.startswith('m.addConstr') and not re.findall(r'<=|>=', line):
            line = re.sub(r'<', r'<=', line)
            line = re.sub(r'>', r'>=', line)
        cleand_code.append(line)
    cleand_code = '\n'.join(cleand_code)
    cleand_code = cleand_code.replace(')m', ')\nm')
    return cleand_code

In [111]:
questions[3]

'A patient in the hospital can take two pills, Pill 1 and Pill 2. Per pill, pill 1 provides 0.2 units of pain medication and 0.3 units of anxiety medication. Per pill, pill 2 provides 0.6 units of pain medication and 0.2 units of anxiety medication. In addition, pill 1 causes 0.3 units of discharge while pill 2 causes 0.1 units of discharge. At most 6 units of pain medication can be provided and at least 3 units of anxiety medication must be provided. How many pills of each should the patient be given to minimize the total amount of discharge?'

In [110]:
code_strs[3]

'x = m.addVar(name="Pill 1", vtype=gp.GRB.CONTINUOUS)\ny = m.addVar(name="Pill 2", vtype=gp.GRB.CONTINUOUS)\nm.setObjective(0.3 * x + 0.1 * y, gp.GRB.MINIMIZE)\nm.addConstr(0.2 * x + 0.6 * y <= 6)\nm.addConstr(0.3 * x + 0.2 * y >= 3)'

In [113]:
execute_code(complement_code(clean_code(code_strs[3])))

2.1428571428571432

In [99]:
def execute_code(code: str) -> float:
    ex_locals = {}
    exec(code, None, ex_locals)
    
    try:
        return ex_locals["m"].objVal
    except Exception as e:
        # print(e)
        return np.inf

In [100]:
pred_answers = []
for i, code_str in enumerate(code_strs):
    try:
        cleaned_code = clean_code(code_str)
        code = complement_code(cleaned_code)
        ans = execute_code(code)
        loguru.logger.info(f"question {i} obtain answer")
        pred_answers.append(ans)
    except Exception as e:
        loguru.logger.error(f"Error for question {i}: {e}")
        pred_answers.append("Error")

2024-09-19 19:21:01.439 | ERROR    | __main__:<module>:10 - Error for question 0: name 'fish_per_sled_dog_trip' is not defined
2024-09-19 19:21:01.441 | INFO     | __main__:<module>:7 - question 1 obtain answer
2024-09-19 19:21:01.443 | INFO     | __main__:<module>:7 - question 2 obtain answer
2024-09-19 19:21:01.444 | INFO     | __main__:<module>:7 - question 3 obtain answer
2024-09-19 19:21:01.447 | INFO     | __main__:<module>:7 - question 4 obtain answer
2024-09-19 19:21:01.448 | INFO     | __main__:<module>:7 - question 5 obtain answer
2024-09-19 19:21:01.450 | INFO     | __main__:<module>:7 - question 6 obtain answer
2024-09-19 19:21:01.452 | INFO     | __main__:<module>:7 - question 7 obtain answer
2024-09-19 19:21:01.454 | INFO     | __main__:<module>:7 - question 8 obtain answer
2024-09-19 19:21:01.456 | INFO     | __main__:<module>:7 - question 9 obtain answer
2024-09-19 19:21:01.458 | INFO     | __main__:<module>:7 - question 10 obtain answer
2024-09-19 19:21:01.460 | INFO  

In [104]:
answers = [a if a != 'None' else str(np.inf) for a in answers]

In [105]:
def mark(pred, real, error: float) -> List[bool]:    
    correct = []
    for p, r in zip(pred, real):
        if p == 'Error':
            continue
        if float(r) != 0:
            if (float(p) == np.inf and float(r) == np.inf) or (abs(float(p) - float(r)) / float(r) < error):
                correct.append(True)
            else:
                correct.append(False)
        else:
            if float(p) < error:
                correct.append(True)
            else:
                correct.append(False)
    return correct

In [108]:
print(pred_answers)

['Error', 5050.0, 125.49295774647888, 2.1428571428571432, 110.0, 45.0, 810.0, 582.0, 1400.0, 36.666666666666664, 118.0, 2650.0, 1500.0, 684000.0, 2.4, 64.0, 125.0, 11275.0, 54.0, 62.5, 66.66666666666667, 160.0, 226.0, 5625.0, 60.0, 1.5, 750.0, 200.0, 735.0, 306.0, 368.0, 34.166666666666664, 603.0, 1000.0, 4347.0, inf, 21.333333333333336, 431.40877598152423, 60.0, inf, 2000.0, 233.33333333333334, inf, 105.0, 80.0, 790.0, 819.0, 1.5, 72.0, 50.0, 350.0, inf, inf, 390.0, 800.0, 571.0, 24.0, 28250.0, 1990.0, 6300.0, 1160.0, 2200.0, 2480.0, 71.0, 14.0, 60.0, 20.0, 75.0, 4000000.0, 465.0, 580.0, 555.0, 120.0, 1760.0, 513.0, 16.0, 67.0, 75.0, 19.0, 236.5, 4190.0, 600.0, 33.5, 160.0, 1215.0, 342.0, 800.0, 19.0, 80.0, 26.0, 327.6595744680851, 1480.0, 310.0, 430.0, 175.0, 960.0, 2500.0, 29.0, 32.0, 78.0, inf, 239.0, 16500.0, 230.0, 24.0, 670.0, inf, 841.0, 9.0, 89.0, 37.0, 23.5, 150000.0, 14.0, 342750.0, 133200.0, 11980.0, 375.0, 3.0, 17.71428571428571, 890.0, 291.6666666666667, 166.6666666666666

In [109]:
print(answers)

['1300', '5050', '125.4929565', '3', '110.0', '45.0', '810.0', '582.0', '1400.0', '36.0', '96', '0', '0.0', '684000.0', '4.0', '64', '125.0', '2500', '98.0', '62.5', '0.0', '160', '226.0', '5625', '60', '1.5', '750.0', '200', '735.0', '360', '368', '0', '603.0', '1000.0', '4347.82612', '899975', '20.8', '440.0', '60', '1266', '2000', '87.5', '1500.0', '105', '68.0', 'inf', '1140', '1.5', '72', '50.0', '350', '833.3333299999999', '0.0', '390', '40', '571.0', '24.0', '0.0', '1990', '6300.0', '1160.0', '2200', '2480', '71.0', '14.0', '60', '20.0', '75.0', '4000000.0', '465.0', '580.0', '555', '120.0', '1970.0', '513', '16.0', '67.0', '75', '19.0', '224.0', '4190.0', '600.0', '33', '150', '540', '342.0', 'inf', '19.0', '80', '26.0', '327.65957199999997', '1480.0', '310.0', '430.0', '175.0', '960', '2500', '29.0', '32', '78', '45', '239.0', '16500', '230.0', '24', '670', '420.0', '841.0', 'inf', '89.0', '37.0', '23', '150000.0', '14', '342750', '133200', '11980', '30000.0', '3.0', '20.0', '

In [107]:
error1, error2 = 1e-2, 1e-4
print(f"Under {error1} error rate: the accuracy is {sum(mark(pred_answers, answers, error=error1)) / len(answers)}")
print(f"Under {error2} error rate: the accuracy is {sum(mark(pred_answers, answers, error=error2)) / len(answers)}")
print(f"Under {1e-6} error rate: the accuracy is {sum(mark(pred_answers, answers, error=1e-6)) / len(answers)}")

Under 0.01 error rate: the accuracy is 0.7152777777777778
Under 0.0001 error rate: the accuracy is 0.7013888888888888
Under 1e-06 error rate: the accuracy is 0.7013888888888888
